<a href="https://colab.research.google.com/github/jgvinholi/DroneDatasetSegmentation/blob/main/OpenDroneDatasetSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Specs

In [ ]:
from psutil import *

In [ ]:
virtual_memory()

In [ ]:
!df -h #disk

In [ ]:
!cat /proc/meminfo

#Save or Retrieve Files

In [ ]:
# Set user
user = 0   #Vinholi
# user = 1   #Debora
# user = 2 # Conta Compartilhada

# Save image to drive
save_img = False

# Define GDrive paths
if user==0:
  base_dir = '/content/drive/MyDrive/ITA/ML/projeto_drone_ita/'
elif user==1:   
  base_dir = '/content/drive/MyDrive/mestrado/PO-233/projeto_drone_ita/'
elif user==2:
  base_dir = '/content/drive/MyDrive/'

working_dir = '/content/SemanticDroneDataset/'
dataset_dir = base_dir+'SemanticDroneDataset/' 
backup_dir = base_dir+'SemanticDroneDataset_backup/'
analise_dir = working_dir+'analise/' 

In [ ]:
# Mount GDrive folder:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get files from Gdrive:
!rsync -avh {dataset_dir} '/content/SemanticDroneDataset' 

sending incremental file list
created directory /content/SemanticDroneDataset
./
.ipynb_checkpoints/
analise/
analise/confusion_matrix_testset.eps
analise/confusion_matrix_trainset.eps
analise/fig_table_attributes_actv_0.eps
analise/fig_table_attributes_actv_16.eps
analise/fig_table_attributes_actv_3.eps
analise/fig_table_attributes_actv_last.eps
analise/overall_metrics_table.csv
analise/overall_metrics_test.pkl
analise/overall_metrics_train.pkl
analise/overall_metrics_val.pkl
analise/pairgrid_table_attributes_actv_last.eps
analise/.ipynb_checkpoints/
analise/detections/
analise/detections/446_gt.png
analise/detections/446_pred.png
analise/detections/447_gt.png
analise/detections/447_pred.png
analise/detections/451_gt.png
analise/detections/451_pred.png
analise/detections/452_gt.png
analise/detections/452_pred.png
analise/detections/454_gt.png
analise/detections/454_pred.png
analise/detections/455_gt.png
analise/detections/455_pred.png
analise/detections/457_gt.png
analise/detections/4

In [ ]:
# Save to Gdrive:
# !rsync -avhr {dataset_dir} {backup_dir} --delete-during 
!rsync -avhr '/content/SemanticDroneDataset/' {dataset_dir} --delete-during

sending incremental file list
analise/overall_metrics_table.csv

sent 31.99K bytes  received 133 bytes  21.41K bytes/sec
total size is 3.71G  speedup is 115,397.82


In [ ]:
import cv2
import glob
def resize(path, interp_type = cv2.INTER_NEAREST, dimsize = (960, 640), ext='.png'):
  for filename in glob.glob(path+ '/*'+ext): #path of raw images
    print(filename)
    img = cv2.imread(filename)
    res = cv2.resize(img, dsize=dimsize, interpolation=interp_type)
    cv2.imwrite(filename, res)

In [ ]:
# resize("/content/SemanticDroneDataset/images", interp_type = cv2.INTER_CUBIC, dimsize = (960, 640), ext='.jpg')

In [ ]:
# resize('/content/SemanticDroneDataset/gt/label_images', interp_type = cv2.INTER_NEAREST, dimsize = (960, 640), ext='.png')

#Includes and Initialization

In [ ]:
!pip install --upgrade line_profiler progressbar2 segmentation-models keras_applications

     |████████████████████████████████| 71kB 3.6MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [ ]:
%env SM_FRAMEWORK=tf.keras
%cd /content/SemanticDroneDataset/
%tensorflow_version 2.x
%load_ext autoreload
%load_ext line_profiler
# %load_ext line_profiler
%autoreload 2

# !git clone https://github.com/bonlime/keras-deeplab-v3-plus.git

from matplotlib import rcParams
from matplotlib.colors import LogNorm
from PIL import Image
from scipy import stats
from scipy import signal
from scipy import ndimage
from matplotlib import cm
from operator import itemgetter
from sklearn.metrics import confusion_matrix
from joblib import Parallel, delayed


import segmentation_models as sm
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import progressbar
import matplotlib
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et
import cv2
import pickle

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.applications.xception import Xception, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import CategoricalCrossentropy


sys.path.insert(1, '/content/SemanticDroneDataset/code')
sys.path.insert(1, '/content/SemanticDroneDataset/keras-deeplab-v3-plus')
from model import Deeplabv3

from basefunctions import *
# Load images and ground truth.
#X_full, img_names = generate_training_var()
#X_full_norm, _ = generate_training_var(True)
#Y_full, Pixels_per_class = generate_onehot_gt()
# Names of attributes.
atributes_names = ['mean', 'standard_deviation', 'skewness', 'kurtosis']
# Matplotlib style
matplotlib.rcParams["font.family"] = "Dejavu Serif"
plt.style.use(['seaborn-colorblind']) 
plt.rcParams["figure.figsize"] = (20, 30)
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2

# try:
#   device_name = os.environ['COLAB_TPU_ADDR']
#   TPU_ADDRESS = 'grpc://' + device_name
#   print('Found TPU at: {}'.format(TPU_ADDRESS))
#   resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
#   tf.config.experimental_connect_to_cluster(resolver) 
#   tf.tpu.experimental.initialize_tpu_system(resolver)
#   strategy = tf.distribute.TPUStrategy(resolver)
# except:
#   pass

env: SM_FRAMEWORK=tf.keras
/content/SemanticDroneDataset
Segmentation Models: using `tf.keras` framework.


#Data Exploration

### Number of images per classes


In [ ]:
# 389.xml EXCLUÍDO, POIS ARQUIVO ESTÁ VAZIO !!!!!!!!!
!rm '/content/SemanticDroneDataset/gt/label_me_xml/389.xml'

path = '/content/SemanticDroneDataset/gt/label_me_xml/'
main_list = []

# Iterate over all .xml files
for filename in os.listdir(path):
  if not filename.endswith('.xml'):
    continue
  fullname = path+filename
  iteration_list = parse_XML(fullname,["object","name"])
  main_list.extend(get_unique(iteration_list))

In [ ]:
# Generate DF
classes_list = np.array(main_list).flatten()
classes_list = classes_list.flatten().flatten()
classes_names, images_per_class = np.unique(classes_list, return_counts=True)

classes_img_df = pd.DataFrame({'Name':classes_names, 'NumImages':images_per_class})
classes_img_df = get_color_code(classes_img_df)
classes_img_df.sort_values('NumImages', inplace=True)


In [ ]:
# Matplotlib style
matplotlib.rcParams["font.family"] = "Dejavu Serif"
#plt.style.use(['seaborn-colorblind']) 
plt.style.use(['seaborn']) 
#plt.rcParams["figure.figsize"] = (20, 30)
plt.rcParams['font.size'] = 14
plt.rcParams['axes.linewidth'] = 2

# Create figure
fig, ax = plt.subplots(figsize=(20, 10))
x = np.arange(len(classes_names))
ax1 = ax.bar(x, height = classes_img_df['NumImages'] )


# Display bar value
for i, v in enumerate(classes_img_df['NumImages']):
    ax.text(i, v+5, str(v), va='center', ha='center', fontweight='bold')

# Edit figure
ax.set_title('Número de imagens que contêm cada classe', fontsize='xx-large', pad=15)
ax.set_ylabel('Número de imagens', fontsize='x-large',labelpad=10)
ax.set_xlabel('Classes', fontsize='x-large',labelpad=10)
ax.set_xticks(x) 
ax.set_xticklabels(classes_img_df['Name'], rotation=45, fontsize='large')
fig.tight_layout()

# Matplotlib general parameters  
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Set color
for bar, color in zip(ax1, classes_img_df['color']):
  try:
    #bar.set_color(np.array(color).astype(float))
    bar.set_color('#9988DD') #'#3388BB', '#EECC55', '#88BB44', '#FFBBBB'
  except:
    pass

#Visualize and save
plt.show
if save_img:
  #plt.savefig(f"{analise_dir}/classe-hist-img.eps")#, format="eps")
  plt.savefig( analise_dir + "classe-hist-img" + ".eps", format="eps") # TODO verificar se o comando tá dando certo aqui

## Feature Visualization

In [ ]:
#def countplot_classes(Pixels_per_class, class_names):
#https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
# Color palette
plt.style.use(['seaborn']) 

# Generate data
npixels_dataset = np.sum(Pixels_per_class)
Pixels_per_class_p100 = 100*Pixels_per_class/npixels_dataset
classes_df = pd.DataFrame( {'Name':class_names, 'NumbPixels':Pixels_per_class, 'Percentage':Pixels_per_class_p100})
classes_df.sort_values('Percentage', inplace=True)

# Create figure
fig, ax = plt.subplots(figsize=(20, 10))
x = np.arange(len(class_names) )
ax1 = ax.bar(x, height = classes_df['Percentage'], color = 'b' )

# Display bar value
for i, v in enumerate(classes_df['Percentage']):
    ax.text(i, v+1, str(round(v,1))+"%", va='center', ha='center', fontweight='bold')

# Edit figure
ax.set_title('Ocorrência das classes a nível de pixel', fontsize='xx-large', pad=15)
ax.set_ylabel('Frequência (%)', fontsize='x-large',labelpad=10)
ax.set_xlabel('Classes', fontsize='x-large',labelpad=10)
ax.set_xticks(x) 
ax.set_xticklabels(classes_df['Name'], rotation=45, fontsize='large')
#ax.bar_label(ax1, padding=3)
fig.tight_layout()
plt.show
print(np.size(classes_df['Name']))

# Set color
for bar in ax1:
    bar.set_color('#9988DD') 


# Matplotlib general parameters  
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

if save_img:
  plt.savefig(f"{analise_dir}/classe-hist-px.eps")
  

In [ ]:
# Create DF 
table_df, table_contents, pixelwise_stats = data_statistics_table(X_full_norm, img_names , act_index = 16)

In [ ]:
table_df, table_contents, pixelwise_stats = data_statistics_table(X_full_norm, img_names , act_index = -1)

In [ ]:
# Load DF from csv

table_df = pd.read_csv('metadata/' + "table_attributes_actv_0" + '.csv')
table_df.corr()

In [ ]:
plot_correlation_heatmap("table_attributes_actv_0")
plot_correlation_heatmap("table_attributes_actv_3")
plot_correlation_heatmap("table_attributes_actv_16")
plot_correlation_heatmap("table_attributes_actv_last")

In [ ]:
# First layer activations (After ReLU):
display_activation(X_full[0], act_index = 3)
# Intermediate layer filters 'block3_sepconv1_act':
display_activation(X_full[0], act_index = 16)
# Last layer activations 'block14_sepconv2_act' (First 256 ones):
display_activation(X_full[0], act_index = -1)

# Data Augmentation

### Definitions


In [ ]:
def random_flipping(Img, Gt):
  # Flip randomly the horizontal and/or vertical axis.
  will_it_be_flipped = np.random.choice([0, 1], 2)
  for i in [0, 1]:
    if will_it_be_flipped[i]:
      Img = np.flip(Img, axis = i)
      Gt = np.flip(Gt, axis = i)
  return Img, Gt

In [ ]:
def cut_window_randomly(Img, Gt, window_size ): # window_size = (vertical_size, horizontal_size)
  imshape = Img.shape
  assert window_size[0] <= imshape[0] and window_size[1] <= imshape[1]
  max_position_start_vert = imshape[0] - window_size[0]
  max_position_start_horiz = imshape[1] - window_size[1]
  # Define window range to be extracted:
  start_vert = np.random.randint(0, max_position_start_vert)
  start_horiz = np.random.randint(0, max_position_start_horiz)
  # Cut window:
  Img = Img[ start_vert:(start_vert+window_size[0]), start_horiz:(start_horiz+window_size[1]) ]
  Gt = Gt[ start_vert:(start_vert+window_size[0]), start_horiz:(start_horiz+window_size[1]) ]
  return Img.astype(np.float32), Gt.astype(np.float32)

In [ ]:
def cut_window_and_rescale_randomly(Img, Gt, scaling_factor_min = -0.1, scaling_factor_max = 0.1, window_size = (320, 480)): 
  '''
    Distort the image by a random scaling factor 'scale', which is in the range (scaling_factor_min, scaling_factor_max).  
    The vertical dimension will be stretched by a factor (1+scale) and the horizontal dimension will be stretched by a factor (1-scale).
  '''
  imshape = Img.shape
  assert window_size[0]*max(abs(scaling_factor_min), abs(scaling_factor_max) ) <= imshape[0] and max(abs(scaling_factor_min), abs(scaling_factor_max) ) <= imshape[1]
  
  scale = np.random.uniform(scaling_factor_min, scaling_factor_max ) # Generate scaling factor by a uniform distribution.
  window_size_before_scaling = ( int(window_size[0]*(1+scale)), int(window_size[1]*(1-scale)) ) # Define the cutting area. This area will then be resized to the desired size 'window_size'.
  # print(window_size_before_scaling)
  Img, Gt = cut_window_randomly(Img, Gt, window_size_before_scaling ) # Cut a random area of the input image with size 'window_size_before_scaling'.
  Img, Gt = resize_window(Img, Gt, window_size)
  return Img.astype(np.float32), Gt.astype(np.float32)

def resize_window(Img, Gt, window_size = (320,480)):
  Img = cv2.resize(Img.astype(np.uint8), dsize=(window_size[1], window_size[0]), interpolation=cv2.INTER_CUBIC ).astype(np.float32) # Resize enlarged image to the desired window size. This creates a stretched (or compressed) image.
  Gt =  cv2.resize(Gt.astype(np.uint8), dsize=(window_size[1], window_size[0]), interpolation=cv2.INTER_NEAREST).astype(np.float32) # Do the same thing to the ground truth. Important to use cv2.INTER_NEAREST to maitain the values of the pixels in the Gt.
  return Img, Gt

In [ ]:
def random_motion_blur(Img, filter_size_min = 3, filter_size_max = 7, augmentation_probability = 0.5): # The size of the filter is proportional to the blur intensity.
  will_blur_be_added = np.random.choice([0, 1], p = [1-augmentation_probability, augmentation_probability]) # Generate a binary RV with P(1) = p and P(0) = 1-p.
  if will_blur_be_added: # If blur should be added in one or both axes:
    which_blur_direction = np.random.choice(['horiz', 'vert', 'diag_pri', 'diag_sec'])
    filter_size = np.random.randint(filter_size_min, filter_size_max) # Set the filter size randomly in the range (3, filter_max_size).
    if filter_size % 2 == 0:
      filter_size += 1 # Make sure filter_size is odd.
    # print("Motion blur filter size = %d." % filter_size)
    
    if which_blur_direction == 'vert': # If the filtering is to be added only in the Y axis.
      # print("The vertical axis will be filtered.")
      filter = np.zeros((filter_size, filter_size)) # Initialize filter
      filter[:, filter_size//2] = 1/filter_size
    if which_blur_direction == 'horiz': # If the filtering is to be added only in the X axis.
      # print("The horizontal axis will be filtered.")
      filter = np.zeros((filter_size, filter_size)) # Initialize filter
      filter[filter_size//2, :] = 1/filter_size
    if which_blur_direction == 'diag_pri':
      # print("The diagonal axis will be filtered.")
      filter = np.diagflat(np.ones(filter_size))*1/filter_size
    if which_blur_direction == 'diag_sec':
      # print("The second diagonal axis will be filtered.")
      filter = np.fliplr( np.diagflat(np.ones(filter_size)) )*1/filter_size
    
    print(filter)
    for chan_ind in [0, 1, 2]: # Convolve every channel with the produced motion blur filter.
      Img[:, :, chan_ind] = signal.fftconvolve(Img[:, :, chan_ind], filter, mode = 'same') 
  return Img.astype(np.float32)

In [ ]:
def modify_hsv_randomly(Img, std = 0.05): # Change Hue, Saturation and Brightness (HSV) randomly based on a normal RV with standard deviation = std*max_channel_value.
  Img = ( cv2.cvtColor(Img, cv2.COLOR_RGB2HSV) ).astype(np.float32) # Convert RGB image to HSV domain.
  rescale_vector = [360, 1, 255] # Maximum values for each HSV channel.
  for hsv_ind in [0, 1, 2]: # Apply transformation to each channel individually.
    rv = np.random.normal(loc = 0, scale = std)*rescale_vector[hsv_ind] # Generate noise value.
    # print(hsv_ind, rv)
    Img[:, :, hsv_ind] = Img[:, :, hsv_ind] + rv # Add to channel.
    Img[:, :, hsv_ind] = np.clip(Img[:, :, hsv_ind], 0, rescale_vector[hsv_ind]) # Clip values bigger than the maximum channel value.
  Img = ( cv2.cvtColor(Img, cv2.COLOR_HSV2RGB) ).astype(np.float32) # Convert image back to RGB domain.
  return Img.astype(np.float32)

In [ ]:
def standardize(Img):
  desc = stats.describe(Img, axis = None)
  Img = (Img - desc.mean )/desc.variance
  return Img

In [ ]:
# Source: https://www.programmersought.com/article/22264609506 . Had to fix some bugs.
def rotate_and_crop_randomly(Img, Gt, angle_min = 0, angle_max = 359, crop = True):
    assert isinstance(Img, np.ndarray) and isinstance(Gt, np.ndarray) and ( isinstance(angle_min + angle_max, int) or isinstance(angle_min + angle_max, float) )
    assert Img.shape == Gt.shape
    crop_image = lambda Img, x0, y0, w, h: Img[x0:x0+w, y0:y0+h, :]  # Define the cutting function and use it for subsequent cutting of black edges
    w, h = Img.shape[:2]
    # The period of the rotation angle is 360°
    angle = int( np.random.uniform(low = angle_min, high = angle_max) )
    angle %= 360
    # print("Rotation angle = %d degrees." % angle)
    # Get the rotated image
    Img = np.array( (Image.fromarray(Img.astype(np.uint8) )).rotate(angle, resample=Image.CUBIC, expand=False, center=None, translate=None, fillcolor=None) )
    Gt = np.array( (Image.fromarray(Gt.astype(np.uint8) )).rotate(angle, resample=Image.NEAREST, expand=False, center=None, translate=None, fillcolor=None) )

    # If you need to remove the black border
    if crop:
        # The equivalent period of the cutting angle is 180°
        angle_crop = angle % 180
        if angle_crop > 90:
            angle_crop = 180 - angle_crop
        # Convert angle to radians
        theta = angle_crop * np.pi / 180
        # Calculate the aspect ratio
        hw_ratio = float(h) / float(w)
        # Calculate the numerator of the cropping side length coefficient
        tan_theta = np.tan(theta)
        numerator = np.cos(theta) + np.sin(theta) * np.tan(theta)

        # Calculate the terms related to the aspect ratio in the denominator
        r = hw_ratio if h > w else 1 / hw_ratio
        # Calculate the denominator term
        denominator = r * tan_theta + 1
        # Final side length coefficient
        crop_mult = numerator / denominator

        # Get crop area
        w_crop = int(crop_mult * w)
        h_crop = int(crop_mult * h)
        x0 = int((w - w_crop) / 2)
        y0 = int((h - h_crop) / 2)
        Img = crop_image(Img, x0, y0, w_crop, h_crop)
        Gt = crop_image(Gt, x0, y0, w_crop, h_crop)
    return Img.astype(np.float32), Gt.astype(np.float32)


In [ ]:
def load_img(img_name):
  # print( str(img_name), images_path)
  if not isinstance(img_name, str):
    img_name = str(img_name, "utf-8")
  Img = np.asarray( image.load_img(images_path + str(img_name) + '.jpg'), dtype=np.float32 )
  # Retrieve ground truth
  Gt_RGB = np.asarray( image.img_to_array( image.load_img(gt_labels_path + img_name + '.png') ),  dtype=np.float32)
  return Img, Gt_RGB

def augment_img(Img, Gt_RGB, n_returns = 1, set_type='train'):
  # 
  if not isinstance(set_type, str):
    set_type = str(set_type, "utf-8")
  assert set_type == 'train' or set_type == 'val'
  # images_path, gt_labels_path, class_colors are global variables.
  wsize = (320,480)
  Img_list = []
  Gt_RGB_list = []
  for i in range(n_returns):
    # Apply transforms:
    # Rotate image before transforms.
    Img_aux = Img
    Gt_RGB_aux = Gt_RGB
    if set_type=='train':
      # Img_aux, Gt_RGB_aux = rotate_and_crop_randomly(Img_aux, Gt_RGB_aux, angle_min = 0, angle_max = 359, crop = True)
      # Img_aux, Gt_RGB_aux = cut_window_and_rescale_randomly(Img_aux, Gt_RGB_aux, window_size = wsize)
      
      Img_aux, Gt_RGB_aux = resize_window(Img_aux, Gt_RGB_aux, window_size = wsize) # 
      Img_aux, Gt_RGB_aux = random_flipping(Img_aux, Gt_RGB_aux)
      Img_aux = modify_hsv_randomly(Img_aux)
    elif set_type=='val':
      Img_aux, Gt_RGB_aux = resize_window(Img_aux, Gt_RGB_aux, window_size = wsize)
    # Standardize 0 mean 1 variance
    Img_aux = standardize(Img_aux)
    Img_list.append(Img_aux.astype(np.float32))
    Gt_RGB_list.append(Gt_RGB_aux)

  Img_list = np.array(Img_list)
  Gt_RGB_list = np.array(Gt_RGB_list)

  # print(Img_list.shape, Gt_RGB_list.shape)
  # Convert GT to one-hot:
  Y = np.zeros( ( Gt_RGB_list.shape[0], Gt_RGB_list.shape[1], Gt_RGB_list.shape[2], nclasses ), dtype = np.float32) # Shape = (nimages, nclasses, npix_vert, npix_horiz ). We substitute the 3 RGB channels for a one-hot vector of dimensions 'nclasses'.
  for class_idx, class_color in enumerate(class_colors):
    coord_pixels_of_a_class = np.logical_and( np.logical_and( ( Gt_RGB_list[:, :, :, 0] == class_color[0] ) ,
                                                             Gt_RGB_list[:, :, :, 1] == class_color[1] ),  Gt_RGB_list[:, :, :, 2] == class_color[2] ) # Retrieve pixel coordinates where class 'class_color' is present.
    # print(coord_pixels_of_a_class.shape, Y[:, :, :, class_idx].shape)
    Y[:, :, :, class_idx][coord_pixels_of_a_class.nonzero()] = 1
  return Img_list, Y.astype(np.float32), Gt_RGB_list.astype(np.uint8)


### Test data augmentation


In [ ]:
# frame1.axes.get_xaxis().set_visible(False)
# frame1.axes.get_yaxis().set_visible(False)
# set_title("Image, origin 'upper'")X, Y, Y_RGB = load_img_and_augment('original/040')
# X_og = np.array( Image.open(images_path + 'original/040' + '.jpg').convert('RGB') )
# plt.imshow( X_og)
# plt.imshow(Y_RGB.astype(np.uint8))

# Model

## Loss function def

In [ ]:
# # ALPHA COMPUTATION

# # Frequencia das classes por pixel
# # Retirei os dados do grafico do relatorio
# # Usei a ordem das classes do .csv

# # TODO verificar se a ordem das saídas da rede é a mesma
# class_list = np.array(
#               ['unlabeled', 'paved-area', 'dirt', 'grass', 'gravel',\
#               'water', 'rocks', 'pool', 'vegetation', 'roof',\
#               'wall', 'window','door', 'fence', 'fence-pole',\
#               'person', 'dog', 'car', 'bicycle', 'tree',\
#               'bald-tree', 'ar-marker', 'obstacle']
# )


# dog e door tem porcentagem 0
# conflicting nao foi contabilizado

freq_list = np.array([[0.001, 0.378, 0.032, 0.20, 0.073,\
             0.022, 0.007, 0.006, 0.071, 0.074,\
             0.027, 0.006, 0.0, 0.01, 0.001,\
             0.011, 0.0, 0.008, 0.002, 0.021,\
             0.013, 0.002, 0.035]]).T

freq_list_argsort = np.flip( np.argsort(freq_list.squeeze()) )

freq_list[(freq_list==0).nonzero()] = 0.001

alpha = 1/np.power(freq_list,1/2)
alpha = (alpha)/np.sum(alpha)
alpha = np.reshape(alpha, (1,1,1,23) )
print(alpha)

freq_list_argsort

[[[[0.1075141  0.00552993 0.01900599 0.00760239 0.01258357 0.02292208
    0.04063651 0.04389245 0.01275958 0.01249826 0.0206911  0.04389245
    0.1075141  0.03399894 0.1075141  0.03241672 0.1075141  0.03801197
    0.07602395 0.0234615  0.02981905 0.07602395 0.0181732 ]]]]


array([ 1,  3,  9,  4,  8, 22,  2, 10,  5, 19, 20, 15, 13, 17,  6, 11,  7,
       21, 18, 14,  0, 12, 16])

In [ ]:
# FOCAL LOSS DEFINITION
def focal_loss(alpha, gamma = 2.0):
  alpha = tf.convert_to_tensor(alpha, dtype=tf.float32)

  def categorical_focal_loss(y_true, y_pred):
    """
    :param y_true: A tensor of the same shape as `y_pred`
    :param y_pred: A tensor resulting from a softmax
    :return: Output tensor.
    """
    tf.cast(y_true, tf.float32)
    tf.cast(y_pred, tf.float32)
    # Clip the prediction value to prevent NaN's and Inf's
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.keras.backend.clip(y_pred, epsilon, 1. - epsilon)

    # Calculate Cross Entropy
    cross_entropy = tf.multiply(-y_true, tf.math.log(y_pred))
    # Calculate Focal Loss
    loss = alpha * tf.pow(1 - y_pred, gamma) * cross_entropy

    # Compute mean loss in mini_batch
    cost = tf.reduce_mean(loss)
    # tf.keras.backend.print_tensor(cost, message='losses')
    return cost
  return categorical_focal_loss



## Unet model def

In [ ]:

# def unet_model(img_size=(320,480,3), num_classes=23):
#     filter_size = 7
#     inputs = Input(shape=img_size)

#     ### [First half of the network: downsampling inputs] ###

#     # Entry block
#     x = Conv2D(32, filter_size, strides=2, padding="same")(inputs)
#     x = BatchNormalization()(x)
#     x = Activation("relu")(x)

#     previous_block_activation = x  # Set aside residual

#     # Blocks 1, 2, 3 are identical apart from the feature depth.
#     for filters in [64, 128, 256]:
#         x = Activation("relu")(x)
#         x = SeparableConv2D(filters, filter_size, padding="same")(x)
#         x = BatchNormalization()(x)

#         x = Activation("relu")(x)
#         x = SeparableConv2D(filters, filter_size, padding="same")(x)
#         x = BatchNormalization()(x)

#         x = MaxPooling2D(3, strides=2, padding="same")(x)

#         # Project residual
#         residual = Conv2D(filters, 1, strides=2, padding="same")(
#             previous_block_activation
#         )
#         x = add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     ### [Second half of the network: upsampling inputs] ###

#     for filters in [256, 128, 64, 32]:
#         x = Activation("relu")(x)
#         x = Conv2DTranspose(filters, filter_size, padding="same")(x)
#         x = BatchNormalization()(x)

#         x = Activation("relu")(x)
#         x = Conv2DTranspose(filters, filter_size, padding="same")(x)
#         x = BatchNormalization()(x)

#         x = UpSampling2D(2)(x)

#         # Project residual
#         residual = UpSampling2D(2)(previous_block_activation)
#         residual = Conv2D(filters, 1, padding="same")(residual)
#         x = add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     # Add a per-pixel classification layer
#     outputs = Conv2D(num_classes, filter_size, activation="softmax", padding="same")(x)

#     # Define the model
#     model = Model(inputs, outputs)
#     model.model_name = "unet"
#     # adam = Adam(learning_rate = 1e-4)
#     # model.compile(optimizer = adam, loss=focal_loss(alpha) )
#     return model



def unet_model(img_size=(320,480,3), num_classes=23):
    filter_size = 3
    inputs = Input(shape=img_size)

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = Conv2D(64, filter_size, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [128, 256, 512]:
        x = Activation("relu")(x)
        x = SeparableConv2D(filters, filter_size, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(filters, filter_size, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [512, 256, 128, 64]:
        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, filter_size, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, filter_size, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        # Project residual
        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = Conv2D(num_classes, filter_size, activation="softmax", padding="same")(x)

    # Define the model
    model = Model(inputs, outputs)
    model.model_name = "unet"
    # adam = Adam(learning_rate = 1e-4)
    # model.compile(optimizer = adam, loss=focal_loss(alpha) )
    return model


# Free up RAM in case the model definition cells were run multiple times

# # Free up RAM in case the model definition cells were run multiple times


## Perform data augmentation def

In [ ]:
def segmentation_generator(img_set, batch_size, batches_per_img, data_set_type):
  # number_of_loads = int( np.ceil( len(img_set)/bulk_size ) )
  while True:
    for img in img_set:
      Img, Gt_RGB = load_img(img)
      for i in range(batches_per_img):
        Img_list, Y_list, _ = augment_img(Img, Gt_RGB, n_returns=batch_size, set_type=data_set_type)
        yield Img_list, Y_list



## Train model

### Train Load Save Model defs

In [ ]:
def train_model(train_index = np.arange(0, 250, 1), validation_index = np.arange(250, 300, 1), n_epochs = 40, model_name="trained_model", model = None, initial_epoch = 0):
  img_path = working_dir + 'images/*.jpg'
  gt_path = working_dir + 'gt/label_images/*.png'
  img_set = [imname.split('/')[-1].split('.')[0] for imname in glob.glob(img_path)]
  img_set = sorted(img_set, key=lambda x: int(x)) 
  train_set = itemgetter(*train_index)(img_set)
  val_set = itemgetter(*validation_index)(img_set)

  batch_size = 5
  batches_per_img = 1
  img_shape = (None, 320, 480, 3)
  gt_shape = (None, 320, 480, 23)
  print('Train: ' +  str(train_set) )
  print('Validation: ' + str(val_set) )

  # es = tf.keras.callbacks.EarlyStopping(
  #       monitor="val_loss",
  #       min_delta=0,
  #       patience=5,
  #       verbose=0,
  #       baseline=None,
  #       restore_best_weights=True,
  #     )
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=dataset_dir + 'trained_models/' + model_name,
    monitor='val_accuracy', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch'
  )

  try:
    train_data = tf.data.Dataset.from_generator( segmentation_generator, args = [ train_set, batch_size, batches_per_img, 'train' ], output_signature=(
                  tf.TensorSpec(shape=img_shape, dtype=tf.float32), tf.TensorSpec(shape=gt_shape, dtype=tf.float32) ) )
    val_data = tf.data.Dataset.from_generator( segmentation_generator, args = [ val_set, 1, 1, 'val' ], output_signature=(
                  tf.TensorSpec(shape=img_shape, dtype=tf.float32), tf.TensorSpec(shape=gt_shape, dtype=tf.float32) ) )
    if model is None:
      # model = unet_model()
      # model = sm.Unet('resnet', classes=23, activation='softmax', encoder_weights='imagenet')
      # model = sm.Unet('inceptionv3', classes=23, activation='softmax', encoder_weights='imagenet')
      # model = sm.Unet('resnet50', classes=23, activation='softmax', encoder_weights='imagenet')
      model = Deeplabv3(weights='cityscapes', input_shape=(img_shape[1], img_shape[2], img_shape[3] ), classes=23, activation='softmax', backbone='xception')
      # model.compile(optimizer = Adam(learning_rate = 1e-3), loss=focal_loss(alpha), metrics=['accuracy'] )
      # cce = tf.keras.losses.CategoricalCrossentropy(
      #         from_logits=False,
      #         label_smoothing=0,
      #         reduction="auto",
      #         name="categorical_crossentropy",
      #     )
      # model.compile(optimizer = Adam(learning_rate = 1e-3), loss=cce, metrics=['accuracy'] )
      model.compile(optimizer = Adam(learning_rate = 1e-3), loss=focal_loss(alpha, 2), metrics=['accuracy'] )
    model.summary()
    model.fit(train_data,
              epochs=n_epochs,
              steps_per_epoch=len(train_set)*batches_per_img,
              validation_data=val_data, 
              validation_steps=len(val_set),
              workers = 4, use_multiprocessing=True,
              callbacks=[model_checkpoint_callback],
              initial_epoch=initial_epoch)
      # history = model.fit(segmentation_generator(train_set, batch_size, batches_per_img, data_set_type='train'), epochs = n_epochs, 
      #             steps_per_epoch = len(train_set)*batches_per_img, # Ceil is used to get the last patches from the image.
      #             validation_data = segmentation_generator(val_set, 1, 1, data_set_type='val'), 
      #             validation_steps = len(val_set), 
      #             workers = 4, use_multiprocessing=True, callbacks=[es] )
    return model
  except KeyboardInterrupt:
    return model


In [ ]:
def save_model(model, model_name, save_to_drive=True):
  assert isinstance(model_name, str)
  model.save(working_dir + 'trained_models/' + model_name )
  if save_to_drive:
    model.save(dataset_dir + 'trained_models/' + model_name )

def load_model(model_name):
  assert isinstance(model_name, str)
  focal_loss_instance = focal_loss(alpha, 2)
  print(working_dir + 'trained_models/' + model_name)
  model = tf.keras.models.load_model(working_dir + 'trained_models/' + model_name, custom_objects={"categorical_focal_loss":focal_loss_instance})
  return model

### Train

In [ ]:
LOAD_WEIGHTS = True
model_to_load = 'trained_model_060621_deeplabv3_nocut'
name_to_save = 'trained_model_060621_deeplabv3_nocut'
last_train_epoch = 0     
total_epochs = 50             

K.backend.clear_session()

if LOAD_WEIGHTS:
  loaded_model = load_model(model_to_load)
  model_trained = train_model(model_name=name_to_save, model=loaded_model, initial_epoch=0, n_epochs=total_epochs) 
else:  
  model_trained = train_model(model_name=name_to_save, initial_epoch=last_train_epoch, n_epochs=total_epochs) 


# NOTES
# 08/Jun Debora:      'trained_model_060621_deeplabv3_nocut'       50/50         val_accuracy: 0.59150


# model_trained = train_model(train_index=[0,1], validation_index=[2,3])
# model_trained = train_model(model_name="trained_model_060621_deeplabv3", model = None) # It was in the 16th epoch
# model_trained = train_model(model_name="trained_model_070621_deeplabv3_cutting", model = model_trained, initial_epoch=39, n_epochs=80) # It was in the 16th epoch


In [ ]:
# save_model(model_trained, 'trained_model_030621')
model_trained = load_model('trained_model_060621_deeplabv3_nocut')

/content/SemanticDroneDataset/trained_models/trained_model_060621_deeplabv3_nocut


## Show detections

In [ ]:
def detect_and_analyse(img_name, model_trained, return_images = False):
  
  Img, Gt = load_img(img_name)
  Img, Y, Gt = augment_img(Img, Gt, 1, 'val')
  n_pix = Img.shape[1]*Img.shape[2]
  Y = np.squeeze(Y)
  Y_pred = model_trained.predict(Img)
  Y_pred_onehot = tf.squeeze( tf.one_hot(tf.argmax(Y_pred, axis=-1), Y.shape[-1]) )
  Y_pred_argmax = np.argmax(Y_pred_onehot, axis=-1)
  Y_pred_rgb = np.zeros( (Y.shape[0], Y.shape[1], 3), dtype = np.uint8 ) # Shape = (nimages, nclasses, npix_vert, npix_horiz ). We substitute the 3 RGB channels for a one-hot vector of dimensions 'nclasses'.
  for class_idx, class_color in enumerate(class_colors):
    coord_pixels_of_a_class = (Y_pred_argmax == class_idx).nonzero()
    Y_pred_rgb[coord_pixels_of_a_class] = class_color
  
  Y_pred_argmax_flat = (Y_pred_argmax).flatten()
  Y_argmax_flat = ( np.argmax(Y, axis=-1) ).flatten()
  confused_matrix = confusion_matrix(Y_argmax_flat, Y_pred_argmax_flat, labels = np.arange(0, 23))
  
  mIOU = np.sum( np.logical_and(Y, Y_pred_onehot), axis = (0, 1) )/np.sum( np.logical_or(Y, Y_pred_onehot), axis = (0, 1) )
  if return_images:
    return Img, Gt, Y_pred_rgb, confused_matrix, mIOU
  else:
    return confused_matrix, mIOU

def multiple_images_metrics(img_index = np.arange(300, 400), model_name = "trained_model_060621_deeplabv3_nocut", model_trained = None):
  if model_trained is None:
    model_trained = load_model(model_name)
  img_path = working_dir + 'images/*.jpg'
  img_set = [imname.split('/')[-1].split('.')[0] for imname in glob.glob(img_path)]
  img_set = np.array(sorted(img_set, key=lambda x: int(x)))[img_index]
  confusion_matrix = []
  mIOU_vector = []
  # with Parallel(n_jobs = 4, verbose=10, backend='threading') as parfor:
  #   results = parfor( delayed(detect_and_analyse)(img_set[i], model_name = model_name, return_images = False) for i in range(len(img_set) ) )
  for i in range(len(img_set) ):
    print("Processing Image " + img_set[i])
    confused_matrix, mIOU = detect_and_analyse(img_set[i], model_trained, return_images = False)
    confusion_matrix.append( confused_matrix )
    mIOU_vector.append(mIOU)
  avg_confusion_matrix = np.sum( np.array(confusion_matrix), axis = 0)
  avg_confusion_matrix = avg_confusion_matrix/np.sum(avg_confusion_matrix+1e-9, axis = 1).reshape(-1,1)
  mIOU_vector = np.array(mIOU_vector)
  mIOU_vector_aux = np.copy(mIOU_vector)
  mIOU_vector_aux[np.isnan( mIOU_vector ) ] = 0
  avg_mIOU_vector = np.sum(mIOU_vector_aux, axis = 0)/np.count_nonzero( np.logical_not( np.isnan(mIOU_vector) ), axis = 0)
  return avg_confusion_matrix, avg_mIOU_vector

def plot_confusion_matrix(confused_matrix, permute_matrix = freq_list_argsort, list_of_classes = class_names, figname = None):
  list_of_classes = np.array(list_of_classes)[permute_matrix]
  print(list_of_classes.shape)
  confused_matrix = confused_matrix[:, permute_matrix][permute_matrix, :]
  confused_matrix[confused_matrix < 1e-5] = np.nan
  print(confused_matrix)
  conf_df = pd.DataFrame(confused_matrix, index = list_of_classes, columns = list_of_classes)
  fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (35,10))
  sns.heatmap(conf_df, annot=True, linewidths=1, linecolor='black', cmap=sns.color_palette("rocket_r", as_cmap=True), norm=LogNorm())
  # sns.heatmap(conf_df, annot=True, linewidths=1, linecolor='black', cmap=sns.color_palette("rocket_r", as_cmap=True))
  if not figname is None:
    plt.savefig( analise_dir + figname + ".eps", format="eps", bbox_inches='tight') # TODO verificar se o comando tá dando certo aqui
  #

def metrics_from_confusion(confusion, save_name = None):
  non_diag_mask = 1 - np.eye( confusion.shape[1])
  confusion_aux = np.copy(confusion)
  confusion_aux[np.isnan(confusion_aux)] = 0
  sum_confusion = np.sum(confusion_aux, axis = 1)
  sum_confusion_vertical = np.sum(confusion_aux, axis = 0)
  recall_class = confusion_aux.diagonal()/sum_confusion
  precision_class = confusion_aux.diagonal()/sum_confusion_vertical
  
  prec_nan_idx = np.isnan(precision_class)
  pclass_aux = np.copy(precision_class)
  pclass_aux = np.delete(pclass_aux, prec_nan_idx)
  freq_list_precision = np.delete(freq_list.squeeze(), prec_nan_idx)

  print(prec_nan_idx, pclass_aux.shape, pclass_aux)
  
  false_negative_rate_class = np.sum( confusion_aux*non_diag_mask, axis = 1 )/sum_confusion
  recall = np.average(recall_class, weights = freq_list.squeeze() )
  false_negative_rate = np.average(false_negative_rate_class, weights = freq_list.squeeze() )
  precision = np.average(pclass_aux, weights = freq_list_precision )
  
  if not save_name is None:
    with open(analise_dir + save_name + '.pkl', 'wb') as f:
      pickle.dump(recall, f)
      pickle.dump(precision, f)
      pickle.dump(false_negative_rate, f)
      pickle.dump(recall_class, f)
      pickle.dump(precision_class, f)
      pickle.dump(false_negative_rate_class, f)
  return recall, precision, false_negative_rate, recall_class, precision_class, false_negative_rate_class

def load_metrics_from_confusion(load_name):
  print(analise_dir + load_name + '.pkl', 'rb')
  with open(analise_dir + load_name + '.pkl', 'rb') as f:
    recall = pickle.load(f)
    precision = pickle.load(f)
    false_negative_rate = pickle.load(f)
    recall_class = pickle.load(f)
    precision_class =  pickle.load(f)
    false_negative_rate_class = pickle.load(f)
  return recall, precision, false_negative_rate, recall_class, precision_class, false_negative_rate_class  

def metrics_setwise(train_index=np.arange(0,250), val_index = np.arange(250, 300), test_index = np.arange(300, 400),
                    model_name = "trained_model_060621_deeplabv3_nocut", model_trained = None):
  if model_trained is None:
    load_model(model_name)
  avg_confusion_matrix_train, avg_mIOU_vector_train = multiple_images_metrics(img_index = train_index, model_trained = model_trained)
  avg_confusion_matrix_val, avg_mIOU_vector_val = multiple_images_metrics(img_index = val_index, model_trained = model_trained)
  avg_confusion_matrix_test, avg_mIOU_vector_test = multiple_images_metrics(img_index = test_index, model_trained = model_trained)
  print( metrics_from_confusion(avg_confusion_matrix_train, save_name = "overall_metrics_train") )
  print( metrics_from_confusion(avg_confusion_matrix_val, save_name = "overall_metrics_val") )
  print( metrics_from_confusion(avg_confusion_matrix_test, save_name = "overall_metrics_test") )


def generate_overall_metrics_table(save_name = "overall_metrics_table", train_pickle_name = "overall_metrics_train", val_pickle_name = "overall_metrics_val", test_pickle_name = "overall_metrics_test", permute_matrix = freq_list_argsort):
  recall_train, precision_train, false_negative_rate_train, recall_class_train, precision_class_train, false_negative_rate_class_train = load_metrics_from_confusion(train_pickle_name)
  recall_val, precision_val, false_negative_rate_val, recall_class_val, precision_class_val, false_negative_rate_class_val = load_metrics_from_confusion(val_pickle_name)
  recall_test, precision_test, false_negative_rate_test, recall_class_test, precision_class_test, false_negative_rate_class_test = load_metrics_from_confusion(test_pickle_name)
  
  recall_class_train, precision_class_train, false_negative_rate_class_train = recall_class_train[permute_matrix], precision_class_train[permute_matrix], false_negative_rate_class_train[permute_matrix]
  recall_class_val, precision_class_val, false_negative_rate_class_val = recall_class_val[permute_matrix], precision_class_val[permute_matrix], false_negative_rate_class_val[permute_matrix]
  recall_class_test, precision_class_test, false_negative_rate_class_test = recall_class_test[permute_matrix], precision_class_test[permute_matrix], false_negative_rate_class_test[permute_matrix]
  
  numpy_table = np.array([
                          np.append( recall_class_train, recall_train),
                          np.append( precision_class_train, precision_train),
                          np.append( false_negative_rate_class_train, false_negative_rate_train),
                          np.append( recall_class_val, recall_val),
                          np.append( precision_class_val, precision_val ),
                          np.append( false_negative_rate_class_val, false_negative_rate_val),
                          np.append( recall_class_test, recall_test),
                          np.append( precision_class_test, precision_test),
                          np.append( false_negative_rate_class_test, false_negative_rate_test)
  ]).transpose()
  print(numpy_table)
  columns_df = np.array(["Recall (Train)", "Precision (Train)", "FNR (Train)", "Recall (Val)", "Precision (Val)",
                       "FNR (Val)", "Recall (Test)", "Precision (Test)", "FNR (Test)"])
  index_df = np.append( np.array(class_names)[permute_matrix], "Total")
  ov_metrics_df = pd.DataFrame(numpy_table, index = index_df, columns = columns_df ).round(decimals=3)
  ov_metrics_df.to_csv(analise_dir + save_name + '.csv', decimal=',')
  print(ov_metrics_df.head())

def show_img_from_array(Img):
  Img = Img.astype(np.uint8)
  fig, ax = plt.subplots(nrows=1, ncols=1)
  ax.imshow(Img)

def detect_and_save(img_name, model_trained = model_trained):
  rsize = (1200, 800)
  Img, Gt, Y_pred_rgb, confused_matrix, mIOU = detect_and_analyse(img_name, model_trained, return_images = True)
  # Img = Image.fromarray(Img.squeeze())
  Gt = Image.fromarray(Gt.squeeze()).resize(rsize, resample=Image.NEAREST)
  Y_pred_rgb = Image.fromarray(Y_pred_rgb.squeeze()).resize(rsize, resample=Image.NEAREST)

  # Img.save(analise_dir + img_name + '_orig.jpg')
  Gt.save(analise_dir + "detections/" + img_name + '_gt.png')
  Y_pred_rgb.save(analise_dir + "detections/" + img_name + '_pred.png')

def detect_multiple_and_save(img_index, model_trained = model_trained):
  img_path = working_dir + 'images/*.jpg'
  img_set = [imname.split('/')[-1].split('.')[0] for imname in glob.glob(img_path)]
  img_set = np.array(sorted(img_set, key=lambda x: int(x)))[img_index]
  for i, img in enumerate(img_set):
    print(img)
    detect_and_save(img, model_trained)

In [ ]:
metrics_setwise(train_index=np.arange(0,250), val_index = np.arange(250, 300), test_index = np.arange(300, 400), model_trained = model_trained)
# metrics_setwise(train_index=np.arange(0,10), val_index = np.arange(10,20), test_index = np.arange(20, 30), model_trained = model_trained)

In [ ]:
generate_overall_metrics_table(save_name = "overall_metrics_table", train_pickle_name = "overall_metrics_train", val_pickle_name = "overall_metrics_val", test_pickle_name = "overall_metrics_test", permute_matrix = freq_list_argsort)

/content/SemanticDroneDataset/analise/overall_metrics_train.pkl rb
/content/SemanticDroneDataset/analise/overall_metrics_val.pkl rb
/content/SemanticDroneDataset/analise/overall_metrics_test.pkl rb
[[0.84850127 0.18941324 0.15149873 0.79786742 0.15561623 0.20213258
  0.77502777 0.13106858 0.22497223]
 [0.61717329 0.27585719 0.38282671 0.59218421 0.2429521  0.40781579
  0.51304511 0.26574149 0.48695489]
 [0.64699631 0.78473065 0.35300369 0.65737437 0.69067805 0.34262563
  0.58970605 0.72095151 0.41029395]
 [0.52792596 0.42082391 0.47207404 0.37336124 0.27312204 0.62663876
  0.41407219 0.24252396 0.58592781]
 [0.45618141 0.36877096 0.54381859 0.36219387 0.30979873 0.63780613
  0.38721231 0.29649279 0.61278769]
 [0.62581823 0.28835896 0.37418177 0.45071007 0.18679235 0.54928993
  0.36349966 0.15223822 0.63650034]
 [0.36464689 0.44219803 0.63535311 0.27146308 0.27674333 0.72853692
  0.24486292 0.30517014 0.75513708]
 [0.71177422 0.31438925 0.28822578 0.44610881 0.17609243 0.55389119
  0.55

In [ ]:
# Detect and save all test set:
detect_multiple_and_save(np.arange(300, 400), model_trained = model_trained)
# detect_and_save('582', model_trained = model_trained)

### Plot Confusion Matrices

In [ ]:
plot_confusion_matrix(avg_confusion_matrix, figname="confusion_matrix_testset")

In [ ]:
plot_confusion_matrix(avg_confusion_matrix_train, figname="confusion_matrix_testset")

In [ ]:
print( np.sum(confused_matrix, axis = 1).reshape(-1,1) + 1e-9)
print(confused_matrix)
print(confused_matrix/np.sum(confused_matrix+1e-9, axis = 1).reshape(-1,1))

In [ ]:
Img, Y_pred_rgb, mIOU, confused_matrix = detect_and_analyse(img_name = '000', model_name = "trained_model_060621_deeplabv3_nocut")
show_img_from_array(Y_pred_rgb)

## History

In [ ]:
# Grafico de acuracia do treinamento

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# Grafico de loss do treinamento

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Função de custo')
plt.ylabel('Custo')
plt.xlabel('Época')
plt.legend(['train', 'val'], loc='upper left')
plt.show()